In [1]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task='irony'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
# twitter-roberta-base-sentiment

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [2]:
# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]
labels

['non_irony', 'irony']

In [7]:
# PT
#model = AutoModelForSequenceClassification.from_pretrained(MODEL)

#text = "Good night 😊"
#text = preprocess(text)
#encoded_input = tokenizer(text, return_tensors='pt')
#output = model(**encoded_input)
#scores = output[0][0].detach().numpy()
#scores = softmax(scores)

# # TF
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

text = "First 2020 depression... yay... 👌🏻 I hate my life"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-irony.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [8]:
ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) irony 0.9808
2) non_irony 0.0192


In [8]:
# esta function retorna los labaels del modelo (negativo, neutral, positivo) 
# recibe un texto de input y retorna string (.. ) 
# es paa usarl con el apply y lambda funciton de pandas 
def return_labels(text):
    encoded_input = tokenizer(text, return_tensors='tf')
    output = model(encoded_input)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    label = labels[ranking[0]]
    return label 

In [9]:
# esta function retorna los probabilidades (para analizar que tan "seguro" esta el modelo de las predicciones) 
# del modelo (negativo, neutral, positivo) 
# recibe un texto de input y retorna string (.. ) 
# es para usar con el apply y lambda function de pandas 
def return_prob(text):
    encoded_input = tokenizer(text, return_tensors='tf')
    output = model(encoded_input)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    return np.sort(scores)[-1]

#### Here we try with our own csv files

In [ ]:
import pandas as pd


In [ ]:
anxiety_df = pd.read_csv('./tweets_2019/depression/hate_depression.csv')
to_model = anxiety_df[['content']].head()

In [ ]:
to_model

In [ ]:
to_model['pre_proccessed'] = to_model['content'].apply(lambda x: preprocess(x)) # mejorar el pre_processing 
# textblob python para hacer tokenizer, lematizing, stemming
# en el paper de este transformers ve como hacen su preprocessing y gtrata de hacer algo igual 
to_model

In [ ]:
to_model['label'] = to_model['pre_proccessed'].apply(lambda x: return_labels(x))
to_model['probability'] = to_model['pre_proccessed'].apply(lambda x: return_prob(x))
to_model